In [3]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

## Lexical features

In [3]:
from core import lexical_features

lexical_features.get_lexical_features('/media/marina/hdd/diploma/data/c++/data/1/1_1.cpp')

[0,
 0,
 -2.665032816018519,
 -3.530030253505124,
 -2.9704144655697013,
 0.15018315018315018,
 15]

In [24]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
%reload_ext autoreload

## Syntactic features

In [20]:
from core.syntactic_features import get_syntactic_features

get_syntactic_features('/media/marina/hdd/diploma/data/c++/data/1/1_1.cpp')

INFO Parsing source file "/media/marina/hdd/diploma/data/c++/data/1/1_1.cpp" ... 
INFO:pygccxml.cxx_parser:Parsing source file "/media/marina/hdd/diploma/data/c++/data/1/1_1.cpp" ... 


[-5.60947179518496, 4.0, 0, 0, 1]

## Cpp keywords

- **module**: cpp_keywords.py
- **input**: filenames (numpy array)
- **output**: keywords frequencies (numpy array)

### Ключевые слова языка C/C++

Могут быть найдены по ссылке в [официальной документации](http://ru.cppreference.com/w/cpp/keyword "cppreference.com")

Частоты ключевых слов языка C/C++:

| float | class | while | namespace | .. |
-----| --- | -------| --- | --- | --- |
file_1| 0.353 | 0.000 | 0.000 | 0.707 | ... |
file_2| 0.401 | 0.213 | 0.000 | 0.708 | ... |
... | ... | ... | ... | ... | ... |
file_n | 0.303 | 0.000 | 0.678 | 0.807 | ... |


In [59]:
%autoreload

from core.cpp_keywords import count_cppkeywords_tf
import pandas as pd
import numpy as np


cpp_keywords_tf = count_cppkeywords_tf(['/media/marina/hdd/diploma/data/c++/data/1/1_1.cpp','/media/marina/hdd/diploma/data/c++/data/1/1_2.cpp'])
print(count_cppkeywords_tf.__doc__)

# table = pd.pivot_table(cpp_keywords_tf)
[print(np.array_split(pd.DataFrame(file), 10)) for file in cpp_keywords_tf]


# import tabletext

# tabletext.to_text(cpp_keywords_tf)

# def remove_zeros(the_list):
#    return [value for value in the_list if value != 0]

# x = [1, 2, 3, 4, 2, 2, 3]
# x = remove_zeros(x, 2)
# print(x)

Returns numpy array of keywords frequencies in each file
[     0
0  0.0
1  0.0
2  0.0
3  0.0
4  0.0
5  0.0
6  0.0
7  0.0
8  0.0,       0
9   0.0
10  0.0
11  0.0
12  0.0
13  0.0
14  0.0
15  0.0
16  0.0
17  0.0,       0
18  0.0
19  0.0
20  0.0
21  0.0
22  0.0
23  0.0
24  0.0
25  0.0
26  0.0,            0
27  0.000000
28  0.000000
29  0.000000
30  0.000000
31  0.000000
32  0.000000
33  0.353553
34  0.000000
35  0.000000,            0
36  0.000000
37  0.000000
38  0.000000
39  0.353553
40  0.000000
41  0.000000
42  0.707107
43  0.000000,       0
44  0.0
45  0.0
46  0.0
47  0.0
48  0.0
49  0.0
50  0.0
51  0.0,            0
52  0.000000
53  0.000000
54  0.000000
55  0.000000
56  0.353553
57  0.000000
58  0.000000
59  0.000000,       0
60  0.0
61  0.0
62  0.0
63  0.0
64  0.0
65  0.0
66  0.0
67  0.0,       0
68  0.0
69  0.0
70  0.0
71  0.0
72  0.0
73  0.0
74  0.0
75  0.0,            0
76  0.353553
77  0.000000
78  0.000000
79  0.000000
80  0.000000
81  0.000000
82  0.000000
83  0.000000]
[    

[None, None]

### widgets

In [6]:
from ipywidgets import interact

def f(x):
    return x

interact(f, x=10);

interact(f, x=True);


@interact(x=True, y=1.0)
def g(x, y):
    return (x, y)

In [12]:
%load_ext autoreload
%autoreload

from ipywidgets import Text, IntSlider, Box, Layout, Label, Dropdown, Textarea, Button
from IPython.display import display

from core.whose_cpp_code import classify_authors, get_filenames

def click(b):
    report = classify_authors(data.value, loops.value)
    print(report['classification_report'])
    print(report['accuracy'])
    print(report['run_time'])

form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

loops = IntSlider(min=0, max=10)
data = Dropdown(options={'students':'/media/marina/hdd/diploma/data/c++/data/', 
                           'GoogleCodeJam':'/media/marina/hdd/diploma/data/c++/data3/', 
                           'GitHub':'/media/marina/hdd/diploma/data/c++/github_data/'})
classifier =  Dropdown(options=['RandomForest', 'GradientBoosting'])
username = Text()
password = Text()

form_items = [
    Box([Label(value='Loops:'), loops], layout=form_item_layout),
    Box([Label(value='Data:'),
         data], 
        layout=form_item_layout),
    Box([Label(value='Classifier:'), classifier], layout=form_item_layout),
#     Box([Label(value='Username:'), username], layout=form_item_layout),
#     Box([Label(value='Password:'), password], layout=form_item_layout),
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
#     border='solid 1px',
    align_items='stretch',
    width='50%'
))

classify_button = Button(description='Classify', tooltip='Click me', icon='check', button_style='success')
classify_button.on_click(click)
# 
display(form, classify_button)
# Text(value='Username')



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


INFO Parsing source file "/media/marina/hdd/diploma/data/c++/data/3/3_6.cpp" ... 
INFO:pygccxml.cxx_parser:Parsing source file "/media/marina/hdd/diploma/data/c++/data/3/3_6.cpp" ... 
INFO Parsing source file "/media/marina/hdd/diploma/data/c++/data/3/3_14.cpp" ... 
INFO:pygccxml.cxx_parser:Parsing source file "/media/marina/hdd/diploma/data/c++/data/3/3_14.cpp" ... 
INFO Parsing source file "/media/marina/hdd/diploma/data/c++/data/3/3_4.cpp" ... 
INFO:pygccxml.cxx_parser:Parsing source file "/media/marina/hdd/diploma/data/c++/data/3/3_4.cpp" ... 
INFO Parsing source file "/media/marina/hdd/diploma/data/c++/data/3/3_11.cpp" ... 
INFO:pygccxml.cxx_parser:Parsing source file "/media/marina/hdd/diploma/data/c++/data/3/3_11.cpp" ... 
INFO Parsing source file "/media/marina/hdd/diploma/data/c++/data/3/3_3.cpp" ... 
INFO:pygccxml.cxx_parser:Parsing source file "/media/marina/hdd/diploma/data/c++/data/3/3_3.cpp" ... 
INFO Parsing source file "/media/marina/hdd/diploma/data/c++/data/3/3_1.cpp"

TypeError: 'module' object is not callable

In [14]:
IntSlider(value=5, min=0, max=10, description='Loops:')

In [11]:
import tqdm

ModuleNotFoundError: No module named 'tqdm'